<a href="https://colab.research.google.com/github/prachiacharya01/SneakPeek/blob/main/Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install implicit
!pip install gmplot
!pip install chart-studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importing Dependencies and Datasets

In [ ]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

import pytz as tz
from datetime import datetime

import gmplot 

In [ ]:
venues_df = pd.read_csv('/content/drive/MyDrive/SneakPeek Project/ALS_dataset/venues.csv')
spot_df = pd.read_csv('/content/drive/MyDrive/SneakPeek Project/ALS_dataset/spot_df')

In [ ]:
venues_df.drop(['Unnamed: 0'], axis=1, inplace=True)
spot_df.drop(['Unnamed: 0'], axis=1, inplace=True)
spot_df.isna().sum()
venues_df.isna().sum()
venues_df = venues_df.drop_duplicates()
spot_df = spot_df.drop_duplicates()

In [ ]:
venues_df

,spotid,spotname,geo-coordinates
0,1391604,Conference House Park,POINT(-74.2523431778 40.5017589436)
1,1391611,Almer G. Russell Pavilion,POINT(-74.2542636395 40.5022647413)
2,3612422,Conference House,POINT(-74.2490418254 40.5000644614)
3,3612431,Billop House,POINT(-74.2490418254 40.5000644614)
4,1391499,Biddle House,POINT(-74.2541456223 40.5054829933)
...,...,...,...
30361,873008,CVS,POINT(-73.8583821667 40.9052244667)
30362,397723,Larchmont Club,POINT(-73.745129 40.9244924667)
30363,1224978,Zitoune,POINT(-73.75237983 40.94017511)
30364,1272266,Stop & Shop,POINT(-73.7475444 40.93326389)


In [ ]:
spot_df

,userid,timestamp,latitude,longitude,spotid,eventType
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847,LIKE
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315,LIKE
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637,LIKE
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516,VIEW
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878,LIKE
...,...,...,...,...,...,...
6442887,196578,2010-06-11T13:32:26Z,51.742988,-0.488065,906885,BOOKMARK
6442888,196578,2010-06-11T13:26:45Z,51.746492,-0.490780,965121,COMMENT CREATED
6442889,196578,2010-06-11T13:26:34Z,51.741916,-0.496729,1174322,VIEW
6442890,196585,2010-10-08T21:01:49Z,50.105516,8.571525,471724,LIKE


# Feature Engineering


Merging above two datasets on the basis of spotid

In [ ]:
merged_df = pd.merge(venues_df,spot_df, on='spotid',how='inner')

In [ ]:
merged_df = merged_df.drop_duplicates()

In [ ]:
merged_df.drop(['geo-coordinates'], axis=1)
merged_df['spotid'] = merged_df['spotid'].astype('category')
merged_df['userid'] = merged_df['userid'].astype('category')
merged_df['s_id'] = merged_df['spotid'].cat.codes
merged_df['u_id'] = merged_df['userid'].cat.codes
merged_df

,spotid,spotname,geo-coordinates,userid,timestamp,latitude,longitude,eventType,s_id,u_id
0,1391604,Conference House Park,POINT(-74.2523431778 40.5017589436),4929,2010-07-05T16:48:52Z,40.501759,-74.252343,BOOKMARK,15105,815
1,1391611,Almer G. Russell Pavilion,POINT(-74.2542636395 40.5022647413),4929,2010-07-05T16:50:26Z,40.502265,-74.254264,COMMENT CREATED,15106,815
2,1391499,Biddle House,POINT(-74.2541456223 40.5054829933),4929,2010-07-05T16:29:01Z,40.505483,-74.254146,LIKE,15104,815
3,403372,Seabra's Armory,POINT(-74.2632088065 40.5039289517),19026,2010-02-22T01:59:37Z,40.503929,-74.263209,COMMENT CREATED,5899,1812
4,403372,Seabra's Armory,POINT(-74.2632088065 40.5039289517),19346,2010-02-27T07:48:40Z,40.503929,-74.263209,VIEW,5899,1836
...,...,...,...,...,...,...,...,...,...,...
114705,891282,Peking House,POINT(-73.8546144962 40.898167491),166694,2010-04-14T17:18:34Z,40.898167,-73.854614,COMMENT CREATED,11389,6619
114706,891282,Peking House,POINT(-73.8546144962 40.898167491),166694,2010-04-07T17:41:28Z,40.898167,-73.854614,BOOKMARK,11389,6619
114707,397723,Larchmont Club,POINT(-73.745129 40.9244924667),378,2010-01-16T23:15:41Z,40.924492,-73.745129,COMMENT CREATED,5822,89
114708,1224978,Zitoune,POINT(-73.75237983 40.94017511),18055,2010-08-28T19:29:19Z,40.940175,-73.752380,BOOKMARK,14099,1773


# Live Events

In [ ]:
def parse_datetime(s):
    tzone = tz.timezone("America/New_York") 
    utc = datetime.strptime(s, '%Y-%m-%dT%H:%M:%SZ')
    return tz.utc.localize(utc).astimezone(tzone)

merged_df['ts'] = merged_df['timestamp'].apply(lambda x: parse_datetime(x))
merged_df = merged_df.drop('timestamp',axis=1,errors='ignore')

#local date and time
merged_df['date']  = merged_df['ts'].astype(object).apply(lambda x : x.date())
merged_df['time']  = merged_df['ts'].astype(object).apply(lambda x : x.time())

merged_df['weekday']  = merged_df['date'].astype(object).apply(lambda x : x.weekday())
merged_df['day']   = merged_df['date'].astype(object).apply(lambda x : x.day)
merged_df['hour']   = merged_df['time'].astype(object).apply(lambda x : x.hour)

merged_df[['ts','date','time','weekday','day','hour']].head()

,ts,date,time,weekday,day,hour
0,2010-07-05 12:48:52-04:00,2010-07-05,12:48:52,0,5,12
1,2010-07-05 12:50:26-04:00,2010-07-05,12:50:26,0,5,12
2,2010-07-05 12:29:01-04:00,2010-07-05,12:29:01,0,5,12
3,2010-02-21 20:59:37-05:00,2010-02-21,20:59:37,6,21,20
4,2010-02-27 02:48:40-05:00,2010-02-27,02:48:40,5,27,2


In [ ]:
merged_df['date'] = merged_df['date'].astype('str')
merged_df['time'] = merged_df['time'].astype('str')
merged_df

,spotid,spotname,geo-coordinates,userid,latitude,longitude,eventType,s_id,u_id,ts,date,time,weekday,day,hour
0,1391604,Conference House Park,POINT(-74.2523431778 40.5017589436),4929,40.501759,-74.252343,BOOKMARK,15105,815,2010-07-05 12:48:52-04:00,2010-07-05,12:48:52,0,5,12
1,1391611,Almer G. Russell Pavilion,POINT(-74.2542636395 40.5022647413),4929,40.502265,-74.254264,COMMENT CREATED,15106,815,2010-07-05 12:50:26-04:00,2010-07-05,12:50:26,0,5,12
2,1391499,Biddle House,POINT(-74.2541456223 40.5054829933),4929,40.505483,-74.254146,LIKE,15104,815,2010-07-05 12:29:01-04:00,2010-07-05,12:29:01,0,5,12
3,403372,Seabra's Armory,POINT(-74.2632088065 40.5039289517),19026,40.503929,-74.263209,COMMENT CREATED,5899,1812,2010-02-21 20:59:37-05:00,2010-02-21,20:59:37,6,21,20
4,403372,Seabra's Armory,POINT(-74.2632088065 40.5039289517),19346,40.503929,-74.263209,VIEW,5899,1836,2010-02-27 02:48:40-05:00,2010-02-27,02:48:40,5,27,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114705,891282,Peking House,POINT(-73.8546144962 40.898167491),166694,40.898167,-73.854614,COMMENT CREATED,11389,6619,2010-04-14 13:18:34-04:00,2010-04-14,13:18:34,2,14,13
114706,891282,Peking House,POINT(-73.8546144962 40.898167491),166694,40.898167,-73.854614,BOOKMARK,11389,6619,2010-04-07 13:41:28-04:00,2010-04-07,13:41:28,2,7,13
114707,397723,Larchmont Club,POINT(-73.745129 40.9244924667),378,40.924492,-73.745129,COMMENT CREATED,5822,89,2010-01-16 18:15:41-05:00,2010-01-16,18:15:41,5,16,18
114708,1224978,Zitoune,POINT(-73.75237983 40.94017511),18055,40.940175,-73.752380,BOOKMARK,14099,1773,2010-08-28 15:29:19-04:00,2010-08-28,15:29:19,5,28,15


In [ ]:
start_time = '11:34:13'
end_time = '16:12:01'	
currentdate = '2010-08-15'
live_events = merged_df[merged_df['date'] == currentdate][merged_df['time']>= start_time][merged_df['time']<=end_time] 
# live_events
live_events['spotname'].drop_duplicates()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



499                          Dunkin' Donuts
2299                        Jacob Riis Park
3394                         EWR Terminal A
3925       EWR Newark Liberty International
5465                                   IKEA
                        ...                
112992                  Nj Transit Broadway
113046          Radburn Station (NJTransit)
113233    Glen Rock Boro Hall (NJT station)
114218                        The Whitehall
114592                     The Village Cafe
Name: spotname, Length: 125, dtype: object

# Strenthening Events

In [ ]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 3.0, 
   'COMMENT CREATED': 4.0,  
}

merged_df['eventStrength'] = merged_df['eventType'].apply(lambda x:event_type_strength[x])

In [ ]:
merged_df

,spotid,spotname,geo-coordinates,userid,latitude,longitude,eventType,s_id,u_id,ts,date,time,weekday,day,hour,eventStrength
0,1391604,Conference House Park,POINT(-74.2523431778 40.5017589436),4929,40.501759,-74.252343,BOOKMARK,15105,815,2010-07-05 12:48:52-04:00,2010-07-05,12:48:52,0,5,12,3.0
1,1391611,Almer G. Russell Pavilion,POINT(-74.2542636395 40.5022647413),4929,40.502265,-74.254264,COMMENT CREATED,15106,815,2010-07-05 12:50:26-04:00,2010-07-05,12:50:26,0,5,12,4.0
2,1391499,Biddle House,POINT(-74.2541456223 40.5054829933),4929,40.505483,-74.254146,LIKE,15104,815,2010-07-05 12:29:01-04:00,2010-07-05,12:29:01,0,5,12,2.0
3,403372,Seabra's Armory,POINT(-74.2632088065 40.5039289517),19026,40.503929,-74.263209,COMMENT CREATED,5899,1812,2010-02-21 20:59:37-05:00,2010-02-21,20:59:37,6,21,20,4.0
4,403372,Seabra's Armory,POINT(-74.2632088065 40.5039289517),19346,40.503929,-74.263209,VIEW,5899,1836,2010-02-27 02:48:40-05:00,2010-02-27,02:48:40,5,27,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114705,891282,Peking House,POINT(-73.8546144962 40.898167491),166694,40.898167,-73.854614,COMMENT CREATED,11389,6619,2010-04-14 13:18:34-04:00,2010-04-14,13:18:34,2,14,13,4.0
114706,891282,Peking House,POINT(-73.8546144962 40.898167491),166694,40.898167,-73.854614,BOOKMARK,11389,6619,2010-04-07 13:41:28-04:00,2010-04-07,13:41:28,2,7,13,3.0
114707,397723,Larchmont Club,POINT(-73.745129 40.9244924667),378,40.924492,-73.745129,COMMENT CREATED,5822,89,2010-01-16 18:15:41-05:00,2010-01-16,18:15:41,5,16,18,4.0
114708,1224978,Zitoune,POINT(-73.75237983 40.94017511),18055,40.940175,-73.752380,BOOKMARK,14099,1773,2010-08-28 15:29:19-04:00,2010-08-28,15:29:19,5,28,15,3.0


# Getting co-ordinates from geo-coordinates from venue_df	

In [ ]:
merged_df = merged_df.drop(["latitude","longitude"],axis=1)
merged_df

,spotid,spotname,geo-coordinates,userid,eventType,s_id,u_id,ts,date,time,weekday,day,hour,eventStrength
0,1391604,Conference House Park,POINT(-74.2523431778 40.5017589436),4929,BOOKMARK,15105,815,2010-07-05 12:48:52-04:00,2010-07-05,12:48:52,0,5,12,3.0
1,1391611,Almer G. Russell Pavilion,POINT(-74.2542636395 40.5022647413),4929,COMMENT CREATED,15106,815,2010-07-05 12:50:26-04:00,2010-07-05,12:50:26,0,5,12,4.0
2,1391499,Biddle House,POINT(-74.2541456223 40.5054829933),4929,LIKE,15104,815,2010-07-05 12:29:01-04:00,2010-07-05,12:29:01,0,5,12,2.0
3,403372,Seabra's Armory,POINT(-74.2632088065 40.5039289517),19026,COMMENT CREATED,5899,1812,2010-02-21 20:59:37-05:00,2010-02-21,20:59:37,6,21,20,4.0
4,403372,Seabra's Armory,POINT(-74.2632088065 40.5039289517),19346,VIEW,5899,1836,2010-02-27 02:48:40-05:00,2010-02-27,02:48:40,5,27,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114705,891282,Peking House,POINT(-73.8546144962 40.898167491),166694,COMMENT CREATED,11389,6619,2010-04-14 13:18:34-04:00,2010-04-14,13:18:34,2,14,13,4.0
114706,891282,Peking House,POINT(-73.8546144962 40.898167491),166694,BOOKMARK,11389,6619,2010-04-07 13:41:28-04:00,2010-04-07,13:41:28,2,7,13,3.0
114707,397723,Larchmont Club,POINT(-73.745129 40.9244924667),378,COMMENT CREATED,5822,89,2010-01-16 18:15:41-05:00,2010-01-16,18:15:41,5,16,18,4.0
114708,1224978,Zitoune,POINT(-73.75237983 40.94017511),18055,BOOKMARK,14099,1773,2010-08-28 15:29:19-04:00,2010-08-28,15:29:19,5,28,15,3.0


In [ ]:
co_ordinates = merged_df["geo-coordinates"].tolist()

In [ ]:
latitude = []
longitude = []
for i in range(len(co_ordinates)):
  geo_coordinates = co_ordinates[i].strip()
  geo_coordinates = geo_coordinates[6:-2]
  geo_coordinates = geo_coordinates.split()
  geo_coordinates = map(float,geo_coordinates)
  pair_final = []
  for i in geo_coordinates:
    pair_final.append(i)
  longitude.append(pair_final[0])
  latitude.append(pair_final[1])

In [ ]:
merged_df['latitude'] = latitude
merged_df['longitude'] = longitude

# sub-part for matrix creation

In [ ]:
merged_df

,spotid,spotname,geo-coordinates,userid,eventType,s_id,u_id,ts,date,time,weekday,day,hour,eventStrength,latitude,longitude
0,1391604,Conference House Park,POINT(-74.2523431778 40.5017589436),4929,BOOKMARK,15105,815,2010-07-05 12:48:52-04:00,2010-07-05,12:48:52,0,5,12,3.0,40.501759,-74.252343
1,1391611,Almer G. Russell Pavilion,POINT(-74.2542636395 40.5022647413),4929,COMMENT CREATED,15106,815,2010-07-05 12:50:26-04:00,2010-07-05,12:50:26,0,5,12,4.0,40.502265,-74.254264
2,1391499,Biddle House,POINT(-74.2541456223 40.5054829933),4929,LIKE,15104,815,2010-07-05 12:29:01-04:00,2010-07-05,12:29:01,0,5,12,2.0,40.505483,-74.254146
3,403372,Seabra's Armory,POINT(-74.2632088065 40.5039289517),19026,COMMENT CREATED,5899,1812,2010-02-21 20:59:37-05:00,2010-02-21,20:59:37,6,21,20,4.0,40.503929,-74.263209
4,403372,Seabra's Armory,POINT(-74.2632088065 40.5039289517),19346,VIEW,5899,1836,2010-02-27 02:48:40-05:00,2010-02-27,02:48:40,5,27,2,1.0,40.503929,-74.263209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114705,891282,Peking House,POINT(-73.8546144962 40.898167491),166694,COMMENT CREATED,11389,6619,2010-04-14 13:18:34-04:00,2010-04-14,13:18:34,2,14,13,4.0,40.898167,-73.854614
114706,891282,Peking House,POINT(-73.8546144962 40.898167491),166694,BOOKMARK,11389,6619,2010-04-07 13:41:28-04:00,2010-04-07,13:41:28,2,7,13,3.0,40.898167,-73.854614
114707,397723,Larchmont Club,POINT(-73.745129 40.9244924667),378,COMMENT CREATED,5822,89,2010-01-16 18:15:41-05:00,2010-01-16,18:15:41,5,16,18,4.0,40.924492,-73.745129
114708,1224978,Zitoune,POINT(-73.75237983 40.94017511),18055,BOOKMARK,14099,1773,2010-08-28 15:29:19-04:00,2010-08-28,15:29:19,5,28,15,3.0,40.940175,-73.752380


In [ ]:
g = merged_df[['u_id','s_id','spotname','eventStrength']]
grouped_df = g.groupby(['u_id','s_id','spotname']).sum().reset_index()

In [ ]:
grouped_df

,u_id,s_id,spotname,eventStrength
0,0,16,New York Stock Exchange,4.0
1,0,46,Times Square,3.0
2,0,76,Empire State Building,1.0
3,0,110,Flatiron Building,1.0
4,0,151,The High Line,2.0
...,...,...,...,...
83743,6891,4104,Liberty Landing Marina,1.0
83744,6891,4320,Brasserie Ruhlmann,2.0
83745,6891,10822,The Hamilton Inn,3.0
83746,6892,49,Ed Sullivan Theatre,1.0


# Most Visited Places

In [ ]:
trending_df = g['spotname'].value_counts(dropna=False)[:20]

In [ ]:
trending_df = pd.DataFrame(trending_df)
trending_df

,spotname
LGA LaGuardia Airport,1679
JFK John F. Kennedy International,1653
Starbucks,1378
Starbucks Coffee,1117
EWR Newark Liberty International,1089
Times Square,1089
Grand Central Terminal,1005
Dunkin' Donuts,575
Madison Square Garden,430
The Museum of Modern Art (MoMA),392


In [ ]:
trending_df.columns = ['Count']

In [ ]:
grouped_df = grouped_df.merge(merged_df[['s_id','latitude','longitude']], how="left",on = "s_id")

In [ ]:
grouped_df = grouped_df.drop_duplicates()
grouped_df[grouped_df['u_id'] == 50]

,u_id,s_id,spotname,eventStrength,latitude,longitude
153236,50,275,EWR Newark Liberty International,3.0,40.691247,-74.179516
154325,50,1941,The Daily Show,2.0,40.767013,-73.993971
154348,50,6394,Starbucks,1.0,40.749671,-73.994035


In [ ]:
merged_df['eventType'].value_counts()

LIKE               28873
VIEW               28717
BOOKMARK           28710
COMMENT CREATED    28410
Name: eventType, dtype: int64

Strength evaluation on the basis of selected event

# Creating Interaction Matrices



In [ ]:
# naming convention: sparse_row_column
sparse_spot_person = sparse.csr_matrix((grouped_df['eventStrength'].astype(float), (grouped_df['s_id'], grouped_df['u_id'])))
sparse_person_spot = sparse.csr_matrix((grouped_df['eventStrength'].astype(float), (grouped_df['u_id'], grouped_df['s_id'])))

# Model

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

In [ ]:
alpha = 15
data = (sparse_spot_person * alpha).astype('double')
# Fit the model

model.fit(data)

  0%|          | 0/50 [00:00<?, ?it/s]

# Function of recommendation

In [ ]:
def final_rec(recommendations_data):
  # filtering recommendations on basis of current user location
  u_lat = 40.502265	
  u_long = -74.254264
  recommendations2 = recommendations_data[(recommendations_data['Latitude']> u_lat-0.1 ) & 
                        (recommendations_data['Latitude']< u_lat+0.1) &
                        (recommendations_data['Longitude']>u_long -0.1) &
                        (recommendations_data['Longitude']<u_long + 0.1)]
  return recommendations2

def recommend(person_id, sparse_person_content, person_vecs, content_vecs, num_contents=10):
    person_interactions = sparse_person_content[person_id,:].toarray()
    person_interactions = person_interactions.reshape(-1) + 1
    person_interactions[person_interactions > 1] = 0

    rec_vector = person_vecs[person_id,:].dot(content_vecs.T).toarray()
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    person_interactions = person_interactions[:rec_vector_scaled.shape[0]]

    # Content already interacted have their recommendation multiplied by zero
    recommend_vector = person_interactions * rec_vector_scaled

    # Sort the indices of the content into order of best recommendations
    spot_idx = np.argsort(recommend_vector)[::-1]#[:num_contents]
    print(len(spot_idx))

    spot_names = []
    scores = []
    latitude = []
    longitude = []
    for idx in spot_idx:
      spot_names.append(grouped_df['spotname'][grouped_df['s_id']==idx].iloc[0])
      latitude.append(grouped_df['latitude'][grouped_df['s_id']==idx].iloc[0])
      longitude.append(grouped_df['longitude'][grouped_df['s_id']==idx].iloc[0])
      scores.append(recommend_vector[idx])
    
    data = {
        'Spotnames': spot_names, 
        'Score': scores,
        'Latitude':latitude,
        'Longitude':longitude
        }
    recommendations = pd.DataFrame(data)
    if recommendations.empty:
        return "recommendations not obtaied"
    else:
        return final_rec(recommendations)

recommenations based on user's previous history

---



In [ ]:
person_vecs = sparse.csr_matrix(model.user_factors)
content_vecs = sparse.csr_matrix(model.item_factors)

person_id = 815

recommendations = recommend(person_id, sparse_person_spot, person_vecs, content_vecs)
print(recommendations)

6893
                       Spotnames     Score   Latitude  Longitude
201                      Wegmans  0.569686  40.548620 -74.297233
338                  Deko Lounge  0.537603  40.465984 -74.278919
560                     JC Penny  0.510633  40.582270 -74.164678
859                      Riffy's  0.493132  40.558389 -74.275876
911                    The Barge  0.490675  40.504053 -74.263670
...                          ...       ...        ...        ...
6466   Perth Amboy Train Station  0.285444  40.509333 -74.273844
6586               The Crossings  0.000000  40.578707 -74.169213
6672          Staten Island Mall  0.000000  40.582470 -74.165920
6769  Apple Store, Staten Island  0.000000  40.582335 -74.165485
6878                   Starbucks  0.000000  40.581790 -74.166861

[63 rows x 4 columns]


user's previous history 

In [ ]:
(grouped_df.loc[grouped_df['u_id'] == 815].sort_values(by=['eventStrength'], ascending=False)[['spotname', 'u_id', 'eventStrength','latitude','longitude']].head(10)).drop_duplicates()

,spotname,u_id,eventStrength,latitude,longitude
1954267,UNIQLO (SoHo),815,24.0,40.723515,-73.998235
1950475,"Apple Store, SoHo",815,18.0,40.725023,-73.999110
1957680,Grand Army Plaza,815,18.0,40.672949,-73.969874
1957218,Ground Support,815,17.0,40.724362,-74.001951
1958732,Glass Shop,815,16.0,40.673630,-73.960679
1955737,Union Square,815,16.0,40.735844,-73.990388
1960165,Brooklyn Bike & Board,815,15.0,40.679914,-73.968211
1943180,Staten Island Ferry,815,14.0,40.700829,-74.013058
1957320,Franklin Park,815,11.0,40.672359,-73.957939
1957782,Scratcher,815,11.0,40.727535,-73.990570


# Ploting points on map

In [ ]:
!pip install chart-studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from chart_studio import plotly as py
import chart_studio
import plotly.figure_factory as ff
import plotly.graph_objs as go

chart_studio.tools.set_credentials_file(
    username='prachi2302',
    api_key='6YA8uypLjx0Rf4SvwFb3'
)

mapbox_access_token = 'pk.eyJ1IjoicHJhY2hpYWNoYXJ5YSIsImEiOiJjbDlndmdocXAwYzcyNDJtb2JrNTkwbWN2In0.Xr04_acZZvKfI08lBAF3pg'

df = recommendations

site_lat = df.Latitude
site_lon = df.Longitude
locations_name = df.Spotnames

# data for the map 
a = pd.Series([40.502265])
b = pd.Series([-74.254264])
data = [
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(242, 177, 172)',
            opacity=0.8
        ),
        text=locations_name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=a,
        lon=b,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(24, 177, 172)',
            opacity=0.8
        ),
        text=locations_name,
        hoverinfo='text'
    )]

layout = go.Layout(
    title='Recommeded Places',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=40.502265	,
            lon=-74.254264
        ),

        pitch=0,
        zoom=10,
        style='dark'
    ),
)
 
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Recommendation')

# END